In [2]:
%pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 3.6 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 8.9 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.13.2
    Uninstalling huggingface-hub-0.13.2:
      Successfully uninstalled huggingface-hub-0.13.2

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import transformers

In [4]:
# Use the HuggingFace transformer library to fine-tune a model on the IMDB library dataset and then evaluate it on the test set. As you do not necessarily have access to a good GPU, and Google Colab is not always providing well, you do not have to fine-tune the model for more than one epoch. There is a fine-tuned model available for steps 2 onward.

# 1. Load the IMDB dataset from the HuggingFace library. You can use the following code to load the dataset:

from datasets import load_dataset
dataset = load_dataset("imdb")

Found cached dataset imdb (/Users/quentinfisch/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# 2. Load the pre-trained model from the HuggingFace library (using distilbert). You can use the following code to load the model:

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizerFast
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [9]:
# 3. Fine-tune the model on the IMDB dataset. You can use the following code to fine-tune the model:

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

dataset.set_format('torch', columns=['text', 'label'])
dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset))
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

training_args = TrainingArguments("test_trainer")
trainer = Trainer(
    model=model, args=training_args, train_dataset=dataset["train"], eval_dataset=dataset["test"]
)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate`: Run `pip install --upgrade accelerate`

In [ ]:
trainer.train()

In [ ]:
# 4. Evaluate the model on the test set. You can use the following code to evaluate the model:

trainer.evaluate()

#### Evaluate the model in term of accuracy on the test data.

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mvonwyl/distilbert-base-uncased-imdb")
model = AutoModelForSequenceClassification.from_pretrained("mvonwyl/distilbert-base-uncased-imdb")

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments("test_trainer")
trainer = Trainer(
    model=model, args=training_args, train_dataset=dataset["train"], eval_dataset=dataset["test"]
)

In [ ]:
trainer.evaluate()

#### For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong.

In [ ]:
# For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong. You can use the following code to get the predictions of the model on the test set:

predictions = trainer.predict(dataset["test"])
predictions = predictions.predictions.argmax(-1)

# wrong predictions
wrong_predictions = []
for i in range(len(predictions)):
    if predictions[i] != dataset["test"][i]["label"]:
        wrong_predictions.append(i)

print(wrong_predictions[:2])

#### What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course? And compared to a recurrent model like an RNN or an LSTM?

The advantages of distilbert model compared to the naive Bayes we implemented is that it takes a lot more information into account. Indeed, the naive Bayes only takes into account the words and their frequency in the sentence. The distilbert model takes into account the context of the words, the order of the words, the meaning of the words, etc. It is a lot more powerful than the naive Bayes model. However, it is also a lot more complex and takes a lot more time to train (naive Bayes is very fast). It is also a lot more difficult to understand and to debug.

The advantages of distilbert model compared to a recurrent model like an RNN or an LSTM is that it is a lot faster to train and to use. Using pre-trained models like distilbert is also a lot easier than training a model from scratch. However, the recurrent models are a lot more flexible and can be used for a lot more tasks than the distilbert model. They can also be used for other languages than English. The recurrent models are also a lot more interpretable than the distilbert model. Indeed, we can see the output of each cell of the RNN and understand what it is doing. It is not the case for the distilbert model.

#### The model only accepts inputs of maximum 512 tokens. Propose and implement a solution that goes around that.

In [ ]:
import torch

# define the sliding window parameters
window_size = 512
stride = 64

# define the input text
input_text = "This is a long input text that needs to be split into smaller parts."

# tokenize the input text
tokens = tokenizer.encode(input_text, add_special_tokens=True)

# initialize the output tensor
output_tensor = torch.zeros((1, 2))

# loop over the input text with a sliding window
for i in range(0, len(tokens), stride):
    # get the current window
    window = tokens[i:i+window_size]
    
    # pad the window if necessary
    if len(window) < window_size:
        window += [tokenizer.pad_token_id] * (window_size - len(window))
    
    # convert the window to a tensor
    window_tensor = torch.tensor(window).unsqueeze(0)
    
    # pass the window through the model
    with torch.no_grad():
        logits = model(window_tensor)[0]
        output_tensor += logits
    
# average the predictions
output_tensor /= (len(tokens) // stride)

# get the predicted class
predicted_class = torch.argmax(output_tensor, dim=1).item()

# get the predicted probability
predicted_probability = torch.softmax(output_tensor, dim=1)[0][predicted_class].item()

# print the results
print(f"Predicted class: {predicted_class}")
print(f"Predicted probability: {predicted_probability}")